In [1]:
import pandas as pd
import json
from pathlib import Path

RAW = "../data/raw/worldbank_pip_2017.csv"   # If you used the 2021 PPP file, change this path
OUT = "../data/processed/pip_uk_percentiles.csv"
META_OUT = "../data/processed/pip_uk_percentiles_meta.json"

Path("../data/processed").mkdir(parents=True, exist_ok=True)

# 1) Load raw PIP data
pip = pd.read_csv(RAW)

# 2) Normalise columns to lower-case for robust filtering
pip.columns = [c.lower() for c in pip.columns]

# 3) Column name guards (PIP exports can vary slightly)
country_col = "country_code" if "country_code" in pip.columns else "country"
level_col   = "reporting_level" if "reporting_level" in pip.columns else "coverage_level"

# 4) Filter to UK (GBR) and national coverage
pip_uk = pip[
    (pip[country_col] == "GBR") &
    (pip[level_col].str.lower() == "national")
].copy()

print("Rows after GBR + national filter:", pip_uk.shape)
pip_uk.head(3)

# 5) Keep essential fields (if any are missing in the raw, this will just keep the available ones)
expected_cols = ["year", "percentile", "avg_welfare", "welfare_type"]
available_cols = [c for c in expected_cols if c in pip_uk.columns]

pip_uk = pip_uk[available_cols].copy()

# 6) Ensure numeric types for percentile and avg_welfare
pip_uk["percentile"] = pd.to_numeric(pip_uk["percentile"], errors="coerce")
pip_uk["avg_welfare"] = pd.to_numeric(pip_uk["avg_welfare"], errors="coerce")
pip_uk = pip_uk.dropna(subset=["percentile", "avg_welfare"]).reset_index(drop=True)

# 7) Sort nicely
pip_uk = pip_uk.sort_values(["year", "percentile"]).reset_index(drop=True)

pip_uk.head(10)

Rows after GBR + national filter: (5400, 10)


,year,percentile,avg_welfare,welfare_type
0,1968,1,5.013687,income
1,1968,2,7.374717,income
2,1968,3,8.018473,income
3,1968,4,8.510337,income
4,1968,5,8.989192,income
5,1968,6,9.357540,income
6,1968,7,9.653367,income
7,1968,8,9.936592,income
8,1968,9,10.215921,income
9,1968,10,10.479103,income


In [2]:
# PIP avg_welfare is daily per person (PPP). Create explicit daily/annual columns.
pip_uk["avg_welfare_daily_ppp"]  = pip_uk["avg_welfare"].astype(float)
pip_uk["avg_welfare_annual_ppp"] = pip_uk["avg_welfare_daily_ppp"] * 365

# Reorder columns for clarity
cols = ["year", "percentile", "avg_welfare_daily_ppp", "avg_welfare_annual_ppp", "welfare_type"]
pip_uk = pip_uk[cols].sort_values(["year", "percentile"]).reset_index(drop=True)

pip_uk.head(10)

,year,percentile,avg_welfare_daily_ppp,avg_welfare_annual_ppp,welfare_type
0,1968,1,5.013687,1829.995590,income
1,1968,2,7.374717,2691.771780,income
2,1968,3,8.018473,2926.742794,income
3,1968,4,8.510337,3106.273122,income
4,1968,5,8.989192,3281.055138,income
5,1968,6,9.357540,3415.501998,income
6,1968,7,9.653367,3523.478984,income
7,1968,8,9.936592,3626.855937,income
8,1968,9,10.215921,3728.811342,income
9,1968,10,10.479103,3824.872674,income


In [3]:
# Basic structure
display(pip_uk.info())
display(pip_uk.head())

# Year & percentile ranges
print("Years:", pip_uk["year"].min(), "→", pip_uk["year"].max())
print("Percentiles:", pip_uk["percentile"].min(), "→", pip_uk["percentile"].max())

# Ensure we have (almost) full percentiles per year
counts = pip_uk.groupby("year")["percentile"].nunique().sort_index()
display(counts.tail(10))
assert counts.min() >= 95, f"Some years have fewer than 95 percentiles: {counts[counts < 95].to_dict()}"

# No duplicate (year, percentile)
dup = pip_uk.duplicated(["year","percentile"]).sum()
print("Duplicate (year, percentile) rows:", dup)
assert dup == 0, "Found duplicated year+percentile entries."

# Welfare type stability (optional but nice)
wtypes_by_year = pip_uk.groupby("year")["welfare_type"].nunique()
mixed = wtypes_by_year[wtypes_by_year > 1]
assert mixed.empty, f"Welfare type changes within a year: {mixed.to_dict()}"

print("Welfare type counts:\n", pip_uk["welfare_type"].value_counts(dropna=False))

<class 'pandas.DataFrame'>
RangeIndex: 5400 entries, 0 to 5399
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    5400 non-null   int64  
 1   percentile              5400 non-null   int64  
 2   avg_welfare_daily_ppp   5400 non-null   float64
 3   avg_welfare_annual_ppp  5400 non-null   float64
 4   welfare_type            5400 non-null   str    
dtypes: float64(2), int64(2), str(1)
memory usage: 211.1 KB


None

,year,percentile,avg_welfare_daily_ppp,avg_welfare_annual_ppp,welfare_type
0,1968,1,5.013687,1829.995590,income
1,1968,2,7.374717,2691.771780,income
2,1968,3,8.018473,2926.742794,income
3,1968,4,8.510337,3106.273122,income
4,1968,5,8.989192,3281.055138,income


Years: 1968 → 2021
Percentiles: 1 → 100


year
2012    100
2013    100
2014    100
2015    100
2016    100
2017    100
2018    100
2019    100
2020    100
2021    100
Name: percentile, dtype: int64

Duplicate (year, percentile) rows: 0
Welfare type counts:
 welfare_type
income    5400
Name: count, dtype: int64


In [4]:
# Save clean dataset
pip_uk.to_csv(OUT, index=False)

# Produce concise metadata for your report
meta = {
    "source_file": RAW,
    "country": "United Kingdom (GBR)",
    "coverage": "national",
    "rows": int(pip_uk.shape[0]),
    "years": [int(pip_uk["year"].min()), int(pip_uk["year"].max())],
    "percentiles_per_year_min": int(pip_uk.groupby("year")["percentile"].nunique().min()),
    "percentiles_per_year_max": int(pip_uk.groupby("year")["percentile"].nunique().max()),
    "welfare_types": pip_uk["welfare_type"].dropna().unique().tolist(),
    "units": {
        "avg_welfare_daily_ppp": "PPP-adjusted income/consumption per person per day",
        "avg_welfare_annual_ppp": "PPP-adjusted income/consumption per person per year"
    }
}

with open(META_OUT, "w") as f:
    json.dump(meta, f, indent=2)

OUT, META_OUT, pip_uk.shape, meta

('../data/processed/pip_uk_percentiles.csv',
 '../data/processed/pip_uk_percentiles_meta.json',
 (5400, 5),
 {'source_file': '../data/raw/worldbank_pip_2017.csv',
  'country': 'United Kingdom (GBR)',
  'coverage': 'national',
  'rows': 5400,
  'years': [1968, 2021],
  'percentiles_per_year_min': 100,
  'percentiles_per_year_max': 100,
  'welfare_types': ['income'],
  'units': {'avg_welfare_daily_ppp': 'PPP-adjusted income/consumption per person per day',
   'avg_welfare_annual_ppp': 'PPP-adjusted income/consumption per person per year'}})